The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name|Genre|
|-----|-----|
|Regal|Hip-Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In order to do so, we will query the chinook database to find out which genres are the most popular in the USA, and choose the corresponding artist to add their album to the store.

In [77]:
%%html
<style>
.nbviewer div.output_area {
  overflow-y: auto;
  max-height: 200px; /* or value of your choosing */
}
</style>

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [16]:
%%sql

SELECT *
FROM sqlite_master
WHERE type in ('table', 'view')

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


In [43]:
%%sql

/*Find number of tracks sold in the USA for each genre*/

/*Create view of sales that had USA as billing country. 
The information includes customer_id, invoice_id, track_id, quantity, track_name, genre_id, and genre_name*/

DROP VIEW IF EXISTS tracks_sold_usa;

CREATE VIEW tracks_sold_usa AS 
    SELECT
        i.customer_id customer_id,
        i.invoice_id,
        il.track_id,
        t.name track_name,
        il.quantity,
        t.genre_id,
        g.name genre_name      
    FROM invoice i
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    INNER JOIN track t ON il.track_id = t.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE billing_country = 'USA';

WITH total_sold AS (
    SELECT SUM(quantity)
    FROM tracks_sold_usa)   


SELECT COUNT(quantity) number_sold, 
       100*COUNT(quantity)/CAST((SELECT SUM(quantity)
                        FROM tracks_sold_usa) AS FLOAT) percentage_sold,
       genre_name
FROM tracks_sold_usa
GROUP BY genre_name
ORDER BY number_sold DESC;

 * sqlite:///chinook.db
Done.
Done.
Done.


number_sold,percentage_sold,genre_name
561,53.37773549000951,Rock
130,12.369172216936251,Alternative & Punk
124,11.798287345385347,Metal
53,5.042816365366318,R&B/Soul
36,3.4253092293054235,Blues
35,3.330161750713606,Alternative
22,2.093244529019981,Pop
22,2.093244529019981,Latin
20,1.9029495718363463,Hip Hop/Rap
14,1.3320647002854424,Jazz


From the table above, we see that the ranking of the 4 genres in the USA in terms of the number of tracks sold is as follows:

|Genre|Rank|Number Sold|Percentage of USA's total sales|
|-----|-----|-----|-----|
|Alternative & Punk|2|130|12.4|
|Blues|5|36|3.43|
|Pop|7|22|2.09|
|Hip Hop/Rap|9|20|1.90|

Based on this alone, we should choose Red Tone, Meteor and the Girls, and Slim Jim Bites to add their albums to the store, as their albums would help us to appeal to the widest possible crowd that is interested in the Alt & Punk, Pop, and Blues scenes respectively.


Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. In order to see if there any differences in the performances of the various sales support agents, we will analyse the purchases of the customers they are in charge of.

To begin, we first find out which of the employees are Sales Support Agents and create a View of them.

In [50]:
%%sql

DROP VIEW IF EXISTS sales_support;

CREATE VIEW sales_support AS
    SELECT *
    FROM employee
    WHERE title = "Sales Support Agent";
    
SELECT * FROM sales_support

 * sqlite:///chinook.db
Done.
Done.
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


Next, we calculate the total dollar each customer has spent on the Chinook store. We do this by checking the total amount they have spent across all their invoices.

In [79]:
%%sql

SELECT c.customer_id, (c.first_name || ' ' || c.last_name) customer_name, SUM(i.total) total_spent, c.support_rep_id, (e.first_name || ' ' || e.last_name) employee_name, e.hire_date
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
INNER JOIN employee e ON c.support_rep_id = e.employee_id
GROUP BY i.customer_id
ORDER BY i.customer_id;

 * sqlite:///chinook.db
Done.


customer_id,customer_name,total_spent,support_rep_id,employee_name,birthdate
1,Luís Gonçalves,108.89999999999998,3,Jane Peacock,1973-08-29 00:00:00
2,Leonie Köhler,82.17,5,Steve Johnson,1965-03-03 00:00:00
3,François Tremblay,99.99,3,Jane Peacock,1973-08-29 00:00:00
4,Bjørn Hansen,72.27000000000001,4,Margaret Park,1947-09-19 00:00:00
5,František Wichterlová,144.54000000000002,4,Margaret Park,1947-09-19 00:00:00
6,Helena Holý,128.7,5,Steve Johnson,1965-03-03 00:00:00
7,Astrid Gruber,69.3,5,Steve Johnson,1965-03-03 00:00:00
8,Daan Peeters,60.38999999999999,4,Margaret Park,1947-09-19 00:00:00
9,Kara Nielsen,37.61999999999999,4,Margaret Park,1947-09-19 00:00:00
10,Eduardo Martins,60.39,4,Margaret Park,1947-09-19 00:00:00


Next, we check the total amount spent by all the customers belonging to each SSA.

In [89]:
%%sql

SELECT support_rep_id ssa_id, employee_name ssa_name, hire_date ssa_hiredate, SUM(total_spent) total_of_ssa
FROM
    (SELECT c.customer_id, (c.first_name || ' ' || c.last_name) customer_name, SUM(i.total) total_spent, c.support_rep_id, (e.first_name || ' ' || e.last_name) employee_name, e.hire_date
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    INNER JOIN employee e ON c.support_rep_id = e.employee_id
    GROUP BY i.customer_id
    ORDER BY i.customer_id)
GROUP BY ssa_id



 * sqlite:///chinook.db
Done.


ssa_id,ssa_name,ssa_hiredate,total_of_ssa
3,Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
4,Margaret Park,2017-05-03 00:00:00,1584.0000000000002
5,Steve Johnson,2017-10-17 00:00:00,1393.92


From the above table, we see that Jane Peacock had the greatest sales numbers, followed by Margaret Park, and finally Steve Johnson. One relationship we can notice is that the further back the SSA's hire date was (and hence the longer they've been with the company), the greater their total. This is an expected observation, as the amount of the time the Sales Support Agents would have had to garner track sales would be longer too.

Next, we want to analyse the sales data for customers from each different country. For this, we do not look at the billing country that the customer has paid via, and will only look at the registered country of the user themselves.

In particular, we will calculate the following data for each country:

1. Total Number of Customers
2. Total Value of Sales
3. Average Value of Sales per Customer
4. Average Order Value



First, we will create a table of the total number of customers and sales for each country. For simplification, we will group all countries that only have one customer under the 'Other' category.

In [166]:
%%sql


DROP VIEW IF EXISTS total_cust_sales_country;

CREATE VIEW total_cust_sales_country AS

WITH country_value AS (
    SELECT c.country, COUNT(DISTINCT c.customer_id) no_of_customers, SUM(inv.total) total_country, COUNT(inv.invoice_id) total_orders,
            CASE
                WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other'
                ELSE c.country
            END AS cat
    FROM invoice inv
    INNER JOIN customer c ON c.customer_id = inv.customer_id
    GROUP BY c.country
    ORDER BY cat)

    /* Sorting */
    SELECT cat country, SUM(no_of_customers) total_num_customers, ROUND(SUM(total_country), 2) total_value_of_sales, SUM(total_orders) total_orders
    FROM 
        (SELECT *, 
             CASE
             WHEN cat = 'Other' THEN 1
             ELSE 0
             END AS sort
         FROM country_value
        )
    GROUP BY cat
    ORDER BY sort;
    
SELECT *
FROM total_cust_sales_country;


 * sqlite:///chinook.db
Done.
Done.
Done.


country,total_num_customers,total_value_of_sales,total_orders
Brazil,5,427.68,61
Canada,8,535.59,76
Czech Republic,2,273.24,30
France,5,389.07,50
Germany,4,334.62,41
India,2,183.15,21
Portugal,2,185.13,29
USA,13,1040.49,131
United Kingdom,3,245.52,28
Other,15,1094.94,147


Next, we check the various statistics that we listed above.

In [167]:
%%sql

SELECT country 'Country', 
        total_num_customers 'Total Number of Customers',
        total_value_of_sales 'Total Value of Sales',
        total_value_of_sales/total_num_customers 'Average Value of Sales per Customer',
        total_value_of_sales/total_orders 'Average Order Value'
        
FROM total_cust_sales_country

 * sqlite:///chinook.db
Done.


Country,Total Number of Customers,Total Value of Sales,Average Value of Sales per Customer,Average Order Value
Brazil,5,427.68,85.536,7.011147540983607
Canada,8,535.59,66.94875,7.047236842105264
Czech Republic,2,273.24,136.62,9.108
France,5,389.07,77.814,7.7814
Germany,4,334.62,83.655,8.161463414634147
India,2,183.15,91.575,8.721428571428572
Portugal,2,185.13,92.565,6.3837931034482756
USA,13,1040.49,80.03769230769231,7.942671755725191
United Kingdom,3,245.52,81.84,8.768571428571429
Other,15,1094.94,72.99600000000001,7.448571428571429


The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

1. Purchase a whole album
2. Purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

We have two edge cases to consider:

1. Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
2. Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

We start by writing a query to check if each invoice is an album purchase or not.

In [196]:
%%sql


SELECT i.invoice_id, il.track_id, t.album_id
FROM invoice i
INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
INNER JOIN track t ON il.track_id = t.track_id
WHERE i.invoice_id = 5;




 * sqlite:///chinook.db
Done.


invoice_id,track_id,album_id
5,1986,163
5,1987,163
5,1988,163
5,1989,163
5,1990,163
5,1991,163
5,1992,163
5,1993,163
5,1994,163
5,1995,163


In [197]:
%%sql

SELECT a.album_id, t.track_id
FROM album a
INNER JOIN track t ON t.album_id = a.album_id
WHERE a.album_id = 163

 * sqlite:///chinook.db
Done.


album_id,track_id
163,1986
163,1987
163,1988
163,1989
163,1990
163,1991
163,1992
163,1993
163,1994
163,1995


In [3]:
%%sql


SELECT CASE
WHEN ((
SELECT il.track_id
FROM invoice i
INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
INNER JOIN track t ON il.track_id = t.track_id
WHERE i.invoice_id = 1

EXCEPT

SELECT t.track_id
FROM album a
INNER JOIN track t ON t.album_id = a.album_id
WHERE a.album_id = 92)

AND

(SELECT t.track_id
FROM album a
INNER JOIN track t ON t.album_id = a.album_id
WHERE a.album_id = 92

EXCEPT

SELECT il.track_id
FROM invoice i
INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
INNER JOIN track t ON il.track_id = t.track_id
WHERE i.invoice_id = 1)) IS NULL THEN 'GAY' ELSE 'NOTGAY' END AS sort


FROM customer

WHERE sort='GAY'

 * sqlite:///chinook.db
Done.


sort


In [29]:
%%sql

DROP VIEW IF EXISTS invoice_album;

CREATE VIEW invoice_album AS
SELECT il.invoice_id, il.track_id invoice_sample_track, t.album_id invoice_sample_album
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
GROUP BY il.invoice_id;

SELECT *
FROM invoice_album
LIMIT 100;

 * sqlite:///chinook.db
Done.
Done.
Done.


invoice_id,invoice_sample_track,invoice_sample_album
1,1158,91
2,3476,322
3,2516,203
4,3448,314
5,1986,163
6,1045,83
7,3477,322
8,2231,183
9,206,21
10,2663,215


In [20]:
%%sql

SELECT invoice_id, invoice_sample_track, invoice_sample_album
FROM (SELECT *
      FROM invoice inv
      INNER JOIN invoice_album ia ON inv.invoice_id = ia.invoice_id)
LIMIT 100;

 * sqlite:///chinook.db
Done.


invoice_id,invoice_sample_track,invoice_sample_album
1,1158,91
2,3476,322
3,2516,203
4,3448,314
5,1986,163
6,1045,83
7,3477,322
8,2231,183
9,206,21
10,2663,215


In [9]:
%%sql

SELECT invoice_id, invoice_sample_track, invoice_sample_album, track_id
FROM (SELECT *
      FROM invoice inv
      INNER JOIN invoice_album ia ON inv.invoice_id = ia.invoice_id
      INNER JOIN track t ON t.album_id = ia.invoice_sample_album
      )
LIMIT 100;




 * sqlite:///chinook.db
Done.


invoice_id,invoice_sample_track,invoice_sample_album,track_id
1,1158,91,1158
1,1158,91,1159
1,1158,91,1160
1,1158,91,1161
1,1158,91,1162
1,1158,91,1163
1,1158,91,1164
1,1158,91,1165
1,1158,91,1166
1,1158,91,1167


In [50]:
%%sql

DROP VIEW IF EXISTS invoices_of_album_purchases;

CREATE VIEW invoices_of_album_purchases AS
SELECT *
FROM invoice_album ia
WHERE (SELECT il.track_id
       FROM invoice_line il
       WHERE il.invoice_id = ia.invoice_id
      
       EXCEPT
       
       SELECT t.track_id
       FROM track t
       WHERE t.album_id = ia.invoice_sample_album) IS NULL
       
       

       AND
       

      (SELECT t.track_id
       FROM track t
       WHERE t.album_id = ia.invoice_sample_album
       
       EXCEPT
       
       SELECT il.track_id
       FROM invoice_line il
       WHERE il.invoice_id = ia.invoice_id) IS NULL;

SELECT *
FROM invoices_of_album_purchases


 * sqlite:///chinook.db
Done.
Done.
Done.


invoice_id,invoice_sample_track,invoice_sample_album
1,1158,91
5,1986,163
23,1,1
24,1146,90
31,2731,221
32,38,6
44,3052,242
46,2639,214
52,2664,216
54,1773,146


In [55]:
%%sql

DROP VIEW IF EXISTS categorised_purchases;

CREATE VIEW categorised_purchases AS
SELECT *,
       CASE
    
        WHEN (SELECT il.track_id
           FROM invoice_line il
           WHERE il.invoice_id = ia.invoice_id

           EXCEPT

           SELECT t.track_id
           FROM track t
           WHERE t.album_id = ia.invoice_sample_album) IS NULL



           AND


          (SELECT t.track_id
           FROM track t
           WHERE t.album_id = ia.invoice_sample_album

           EXCEPT

           SELECT il.track_id
           FROM invoice_line il
           WHERE il.invoice_id = ia.invoice_id) IS NULL

        THEN 'purchased_album'
        ELSE 'purchased_tracks'
        END AS 'purchase_category'
        
FROM invoice_album ia;

 * sqlite:///chinook.db
Done.
Done.


[]

In [70]:
%%sql

SELECT purchase_category, 
number_of_invoices, 
number_of_invoices/(SELECT CAST(COUNT(*) AS FLOAT) FROM categorised_purchases)*100  pct_of_total_invoices
FROM (
SELECT purchase_category, COUNT(*) number_of_invoices
FROM categorised_purchases
GROUP BY purchase_category
)


 * sqlite:///chinook.db
Done.


purchase_category,number_of_invoices,pct_of_total_invoices
purchased_album,114,18.566775244299674
purchased_tracks,500,81.43322475570032


From our table above, we find that a large majority of purchases in the Chinook Store are purchases of individual tracks (81%), compared to the small minority of album purchases (19%). As such, based on this alone, I recommend that we could look at changing our purchasing strategy to focus on purchasing only popular tracks from record companies. However, this could be discussed after further analysis on the amounts that full album purchases bring in vs individual track purchases.